# Weather Information Agent

The Weather Information Agent is designed to interpret natural language weather-related queries, extract the location, fetch real-time weather data, and generate a user-friendly response.

- Location Extraction: Uses OpenAI API to determine the location from user queries.

- Weather Data Retrieval: Fetches real-time weather details such as temperature, humidity, wind speed, and cloud coverage using Weatherstack API.

- Natural Language Response: Generates conversational weather descriptions using OpenAI’s language model.

- Comprehensive Query Processing: Ensures an end-to-end flow from understanding the user’s question to delivering an insightful weather report.

This agent provides an interactive and seamless weather update experience for users.

### 1. Install necessary packages

In [14]:
!pip install openai python-dotenv nest_asyncio

### 2. Import Dependencies

In [6]:
import os
import json
import requests
import asyncio
from dotenv import load_dotenv
from agents import Agent
from agents.mcp import MCPServer
from agents import function_tool
import nest_asyncio
nest_asyncio.apply()

### 3. Load the environment variables

In [7]:
load_dotenv()

# set up API keys
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')

### 4. Weather Agent Class
The WeatherAgent class is a Python implementation that provides weather information for a given location using natural language processing. It combines the OpenAI API for language understanding and the Weatherstack API for retrieving weather data.

### Key Components
#### Initialization
- The class requires API keys for both Weatherstack and OpenAI services
- These keys are used to authenticate API requests
#### Methods 

**1. call_openai_api**
- Makes requests to OpenAI's API
- Sends messages in a specific format required by OpenAI
- Returns the generated text response or None if an error occurs
- Handles API errors and exceptions gracefully 

**2. extract_location**
- Uses OpenAI to extract location information from a natural language query
- Provides specific instructions to the AI to focus only on location extraction
- Returns the extracted location name 

**3. get_weather_data**
- Fetches current weather data from Weatherstack API for a specified location
- Processes the API response to extract relevant weather information
- Returns a structured dictionary with key weather metrics including:
  - Location name and country
  - Temperature and "feels like" temperature
  - Humidity and wind speed
  - Weather description and cloud cover

**4. generate_weather_response**
- Creates a natural language description of weather conditions
- Uses OpenAI to convert structured weather data into conversational text
- Returns a friendly, informative weather description 

**5. process_weather_query**
- Orchestrates the entire weather information process
- Extracts location from user query
- Retrieves weather data for that location
- Generates a natural language response
- Handles errors at each step with appropriate fallback messages

In [8]:
class WeatherAgent:
    def __init__(self, weather_api_key, openai_api_key):
        """
        Initialize the Weather Agent with API keys.
        
        Args:
            weather_api_key (str): API key for weather service
            openai_api_key (str): API key for OpenAI
        """
        self.weather_api_key = weather_api_key
        self.openai_api_key = openai_api_key
        self.conversation_context = {
            "last_location": None,
            "last_weather_data": None,
            "query_history": []
        }
    
    def call_openai_api(self, messages, model="gpt-3.5-turbo"):
        """
        Call OpenAI API directly using requests.
        
        Args:
            messages (list): List of message objects
            model (str): Model to use
            
        Returns:
            str: Response content
        """
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.openai_api_key}"
        }
        
        # Create the payload
        payload = {
            "model": model,
            "messages": messages,
            "temperature": 0.7
        }
        
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers=headers,
                json=payload
            )
            
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"].strip()
            else:
                print(f"Error calling OpenAI API: {response.status_code}")
                print(response.text)
                return None
        except Exception as e:
            print(f"Exception when calling OpenAI API: {e}")
            return None

    def extract_location(self, user_query):
        """
        Extract location from user query using OpenAI.
        
        Args:
            user_query (str): Natural language weather query
        
        Returns:
            str: Extracted location name or None if no location is found
        """
        # Add context from previous queries
        context_prompt = ""
        if self.conversation_context["last_location"]:
            context_prompt = f"Previous location mentioned was {self.conversation_context['last_location']}. "
        
        # Create system message with improved instructions
        system_content = (
            "You are a location extraction assistant. Extract the specific city or location from the given query. "
            "If the query doesn't mention a specific location but refers to a previous location, use that previous location. "
            "If the query is asking a comparative question like 'Is it warmer than yesterday?' or a future question like "
            "'Will I need an umbrella tomorrow?' without specifying a location, respond with 'USE_PREVIOUS_LOCATION'. "
            "If no location is mentioned and there's no previous context, respond with 'NO_LOCATION'."
        )
        
        messages = [
            {"role": "system", "content": system_content},
            {"role": "user", "content": f"{context_prompt}Extract the location from this query: '{user_query}'."}
        ]
        
        try:
            location = self.call_openai_api(messages)
            
            # Handle special responses
            if location == "USE_PREVIOUS_LOCATION":
                return self.conversation_context["last_location"]
            elif location == "NO_LOCATION":
                return None
            
            # Store the location in context
            self.conversation_context["last_location"] = location
            return location
        except Exception as e:
            print(f"Error extracting location: {e}")
            return None
    
    def get_weather_data(self, location):
        """
        Fetch weather data for a given location using Weatherstack API.
        
        Args:
            location (str): City name or location identifier
        
        Returns:
            dict: Processed weather information
        """
        base_url = "http://api.weatherstack.com/current"
        params = {
            'access_key': self.weather_api_key,
            'query': location,
            'units': 'm'  # Metric units
        }
        
        try:
            response = requests.get(base_url, params=params)
            data = response.json()
            
            # Check for errors in the response
            if 'error' in data:
                print(f"Weather API error: {data['error']['info']}")
                return None
            
            # Extract relevant weather information
            weather_info = {
                'location': data['location']['name'],
                'country': data['location']['country'],
                'temperature': data['current']['temperature'],
                'feels_like': data['current']['feelslike'],
                'humidity': data['current']['humidity'],
                'description': data['current']['weather_descriptions'][0] if data['current']['weather_descriptions'] else 'No description available',
                'wind_speed': data['current']['wind_speed'],
                'cloudiness': data['current']['cloudcover']
            }
            
            # Store the weather data in context
            self.conversation_context["last_weather_data"] = weather_info
            return weather_info
        
        except Exception as e:
            print(f"Error fetching weather data: {e}")
            return None
    
    def generate_weather_response(self, weather_data, user_query):
        """
        Generate a natural language response for weather data.
        
        Args:
            weather_data (dict): Weather information
            user_query (str): Original user query
        
        Returns:
            str: Descriptive weather response
        """
        # Add context for comparative or future queries
        context_prompt = ""
        if "yesterday" in user_query.lower() or "warmer" in user_query.lower() or "colder" in user_query.lower():
            context_prompt = "This is a comparative question about temperature. "
        elif "tomorrow" in user_query.lower() or "umbrella" in user_query.lower() or "rain" in user_query.lower():
            context_prompt = "This is a question about future weather or precipitation. "
        
        messages = [
            {"role": "system", "content": f"You are a friendly weather narrator. Create a conversational weather description. {context_prompt}"},
            {"role": "user", "content": f"Create a friendly, informative weather description for this query: '{user_query}' using this data: {weather_data}"}
        ]
        
        try:
            response = self.call_openai_api(messages)
            return response
        except Exception as e:
            print(f"Error generating weather response: {e}")
            return None
    
    def process_weather_query(self, user_query):
        """
        Process a complete weather query from extraction to response.
        
        Args:
            user_query (str): Natural language weather query
        
        Returns:
            str: Comprehensive weather information response
        """
        # Store query in history
        self.conversation_context["query_history"].append(user_query)
        
        # Extract location
        location = self.extract_location(user_query)
        if not location:
            if self.conversation_context["last_location"]:
                # Use the last location if available
                location = self.conversation_context["last_location"]
                print(f"Using previous location: {location}")
            else:
                return "I need to know which location you're asking about. Could you specify a city or place?"
        
        # Get weather data
        weather_data = self.get_weather_data(location)
        if not weather_data:
            return f"Sorry, I couldn't retrieve weather data for {location}."
        
        # Generate natural language response
        weather_response = self.generate_weather_response(weather_data, user_query)
        return weather_response or "I encountered an issue generating the weather description."

### 5. Create the MCP server

The WeatherMCPServer class is an implementation of the Model-Control-Protocol (MCP) server that provides weather information services. Here's a description of its functionality:

This class serves as a bridge between the Agent SDK framework and the custom WeatherAgent implementation. It handles the registration, discovery, and execution of weather-related tools that can be used by AI agents.

Key components:

1. **Initialization** : Creates a server instance with a weather agent, either using a provided agent or creating a new one with the configured API keys.
2. **Server Properties** : Provides a unique name identifier ("weather_server") for the MCP framework to reference.
3. **Connection Management** : Implements required connection methods ( connect and cleanup ) that handle server lifecycle, though in this implementation they're simplified since no actual external connection is needed.
4. **Tool Execution** : The *call_tool* method routes incoming tool requests to the appropriate handler - in this case, it processes weather queries by delegating to the WeatherAgent .
5. **Tool Registration** : The list_tools method exposes the available tools to the agent framework by creating a FunctionTool that describes the weather query capability, including its parameters and requirements.

In [9]:
# MCP Server implementation for weather
class WeatherMCPServer(MCPServer):
    def __init__(self, weather_agent=None):
        super().__init__()
        self.weather_agent = weather_agent or WeatherAgent(
            weather_api_key=WEATHER_API_KEY, 
            openai_api_key=OPENAI_API_KEY
        )
    
    @property
    def name(self):
        """Return the name of the server."""
        return "weather_server"
    
    async def connect(self):
        """Connect to the server."""
        # No actual connection needed for this example
        return True
    
    async def cleanup(self):
        """Clean up resources."""
        # No cleanup needed for this example
        pass
    
    async def call_tool(self, tool_name, tool_params):
        """Call a tool by name with parameters."""
        if tool_name == "weather_tool":
            return self.weather_agent.process_weather_query(tool_params["query"])
        raise ValueError(f"Unknown tool: {tool_name}")
    
    async def list_tools(self):
        """List available tools."""
        # Create a weather tool using function_tool module
        weather_tool = function_tool.FunctionTool(
            name="weather_tool",
            description="Get current weather information for a location",
            function=lambda params: self.weather_agent.process_weather_query(params["query"]),
            parameters={
                "query": {
                    "type": "string",
                    "description": "The location or weather query to process"
                }
            },
            required=["query"]
        )
        return [weather_tool]

### 6. Setup and run the weather agent

The setup_and_run_agent function is designed to process weather queries using the WeatherAgent directly, bypassing the Agent SDK framework due to compatibility issues. Here's a description of its functionality:

This function serves as a simplified interface for processing weather queries when the standard Agent SDK approach encounters issues. It:

1. Creates a WeatherMCPServer instance (though this isn't actually used in the function)
2. Initializes a new WeatherAgent with the necessary API keys
3. Directly processes the user's query through the agent's process_weather_query method
4. Returns the generated response

The *persistent_weather_agent* declaration that follows creates a single, long-lived instance of the WeatherAgent that maintains conversation context across multiple queries. This is crucial for handling follow-up questions that rely on previous context, such as "How about in London?" or "Is it warmer than yesterday?"

In [ ]:
# Function to set up and run the agent with MCP
async def setup_and_run_agent(user_query):
    # Create the MCP server with your weather tool
    weather_server = WeatherMCPServer()
    

    weather_agent = WeatherAgent(
        weather_api_key=WEATHER_API_KEY,
        openai_api_key=OPENAI_API_KEY
    )
    
    # Process the query directly using our weather agent
    response = weather_agent.process_weather_query(user_query)
    
    # Return the response
    return response
persistent_weather_agent = WeatherAgent(
    weather_api_key=WEATHER_API_KEY,
    openai_api_key=OPENAI_API_KEY
)


In [11]:
# Function to handle running the agent in different environments
def get_weather_from_agent(query):
    """
    Process a weather query using our weather agent directly
    """
    # Use the persistent agent to maintain context between queries
    return persistent_weather_agent.process_weather_query(query)

In [12]:
# Process a series of queries to demonstrate functionality
queries = [
    "What's the weather like in New York today?",
    "How about in London?",
    "Is it warmer than yesterday?",
    "Will I need an umbrella tomorrow?"
]

# Process each query using the MCP agent
for i, query in enumerate(queries):
    print(f"\nQuery {i+1}: {query}")
    result = get_weather_from_agent(query)
    print(result)


Query 1: What's the weather like in New York today?
Hey there! If you're stepping out in New York today, you can expect partly cloudy skies with a temperature of 6°C, but it might feel a bit cooler at 1°C due to the wind chill. The humidity is at a comfortable 28%, so it won't feel too muggy. The wind is blowing at a brisk pace of 35 km/h, so hold onto your hat! The cloud coverage is at 50%, so you might catch a glimpse of the sun peeking through the clouds. Enjoy your day in the Big Apple!

Query 2: How about in London?
Oh, London is looking lovely today! The current temperature is a comfortable 12 degrees Celsius, but it feels slightly cooler at 11 degrees with a gentle breeze of 10 km/h. The humidity is at a comfortable 44%, so it's not too muggy. The skies are clear with no clouds in sight, making it a perfect day to go for a stroll or enjoy some outdoor activities. Don't forget to grab a light jacket before heading out to enjoy the beautiful weather in the United Kingdom's capita